In [1]:
pip install streamlit


  Obtaining dependency information for streamlit from https://files.pythonhosted.org/packages/88/34/0751fd391fe90c8a90e70ef267080e0d461a2bf00c9e84c5a39d3389470d/streamlit-1.34.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for altair<6,>=4.0 from https://files.pythonhosted.org/packages/46/30/2118537233fa72c1d91a81f5908a7e843a6601ccc68b76838ebc4951505f/altair-5.3.0-py3-none-any.whl.metadata
  Obtaining dependency information for blinker<2,>=1.0.0 from https://files.pythonhosted.org/packages/bb/2a/10164ed1f31196a2f7f3799368a821765c62851ead0e630ab52b8e14b4d0/blinker-1.8.2-py3-none-any.whl.metadata
  Obtaining dependency information for cachetools<6,>=4.0 from https://files.pythonhosted.org/packages/fb/2b/a64c2d25a37aeb921fddb929111413049fc5f8b9a4c1aefaffaafe768d54/cachetools-5.3.3-py3-none-any.whl.metadata
  Obtaining dependency information for gitpython!=3.1.19,<4,>=3.0.7 from https://files.pythonhosted.org/packages/e9/bd/cc3a402a6439c15c3d4294333e13042b915bbeab54edc

In [3]:
import streamlit as st

In [1]:
import numpy as np
import pandas as pd
import pickle
import streamlit as st
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import plotly.express as px
import plotly.graph_objects as go

# Load the saved model and preprocessing objects
loaded_model = pickle.load(open("trained_model.sav", "rb"))
with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)
with open('pca.pkl', 'rb') as f:
    pca = pickle.load(f)

# Function for prediction
def fraud_prediction(input_data):
    try:
        input_data_as_numpy_array = np.asarray(input_data)
        input_data_reshaped = input_data_as_numpy_array.reshape(len(input_data_as_numpy_array), -1)
        input_data_scaled = scaler.transform(input_data_reshaped)
        input_data_pca = pca.transform(input_data_scaled)
        prediction = loaded_model.predict(input_data_pca)
        result = ['Non-fraud' if pred == 0 else 'Fraud' for pred in prediction]
        return result
    except Exception as e:
        return str(e)

def main():
    html_temp = """
    <div style="background-color:tomato;padding:10px">
    <h2 style="color:white;text-align:center;">Streamlit Ethereum Fraud Detection ML App</h2>
    </div>
    """
    st.markdown(html_temp, unsafe_allow_html=True)

    mode = st.radio("Select Mode", ("Batch Prediction", "Real-time Prediction"))

    if mode == "Batch Prediction":
        uploaded_file = st.file_uploader("Upload CSV file", type=["csv"])

        if uploaded_file is not None:
            input_df = pd.read_csv(uploaded_file)

            if input_df.isnull().values.any():
                st.warning("Error occured. Please check your file.")
                return

            input_data = input_df.iloc[:, 2:].values.tolist()  # Exclude the first two columns (address and flag)
            true_labels = input_df['flag'].values

            if st.button("Predict"):
                with st.spinner("Processing..."):
                    predictions = fraud_prediction(input_data)
                    predicted_labels = [0 if pred == 'Non-fraud' else 1 for pred in predictions]

                    accuracy = accuracy_score(true_labels, predicted_labels)
                    precision = precision_score(true_labels, predicted_labels)
                    recall = recall_score(true_labels, predicted_labels)
                    f1 = f1_score(true_labels, predicted_labels)

                st.success("Predictions Complete!")
                st.write(f"**Accuracy:** {accuracy * 100:.2f}%")
                st.write(f"**Precision:** {precision:.2f}")
                st.write(f"**Recall:** {recall:.2f}")
                st.write(f"**F1 Score:** {f1:.2f}")

                input_df['Prediction'] = predictions
                st.write(input_df)

                fig = px.histogram(input_df, x='Prediction', title='Fraud vs Non-Fraud Transactions')
                st.plotly_chart(fig)

                st.download_button(
                    label="Download Predictions Result",
                    data=input_df.to_csv().encode('utf-8'),
                    file_name='predictions.csv',
                    mime='text/csv'
                )

    elif mode == "Real-time Prediction":
        minTimeBetweenSentTnx = st.text_input("minTimeBetweenSentTnx", "Type Here")
        maxTimeBetweenSentTnx = st.text_input("maxTimeBetweenSentTnx", "Type Here")
        avgTimeBetweenSentTnx = st.text_input("avgTimeBetweenSentTnx", "Type Here")
        minTimeBetweenRecTnx = st.text_input("minTimeBetweenRecTnx", "Type Here")
        maxTimeBetweenRecTnx = st.text_input("maxTimeBetweenRecTnx", "Type Here")
        avgTimeBetweenRecTnx = st.text_input("avgTimeBetweenRecTnx", "Type Here")
        lifetime = st.text_input("lifetime", "Type Here")
        sentTransactions = st.text_input("sentTransactions", "Type Here")
        receivedTransactions = st.text_input("receivedTransactions", "Type Here")
        createdContracts = st.text_input("createdContracts", "Type Here")
        numUniqSentAddress = st.text_input("numUniqSentAddress", "Type Here")
        numUniqRecAddress = st.text_input("numUniqRecAddress", "Type Here")
        minValSent = st.text_input("minValSent", "Type Here")
        maxValSent = st.text_input("maxValSent", "Type Here")
        avgValSent = st.text_input("avgValSent", "Type Here")
        minValReceived = st.text_input("minValReceived", "Type Here")
        maxValReceived = st.text_input("maxValReceived", "Type Here")
        avgValReceived = st.text_input("avgValReceived", "Type Here")
        totalTransactions = st.text_input("totalTransactions", "Type Here")
        totalEtherSent = st.text_input("totalEtherSent", "Type Here")
        totalEtherReceived = st.text_input("totalEtherReceived", "Type Here")
        totalEtherSentContracts = st.text_input("totalEtherSentContracts", "Type Here")
        totalEtherBalance = st.text_input("totalEtherBalance", "Type Here")
        activityDays = st.text_input("activityDays", "Type Here")
        dailyMax = st.text_input("dailyMax", "Type Here")
        ratioRecSent = st.text_input("ratioRecSent", "Type Here")
        ratioSentTotal = st.text_input("ratioSentTotal", "Type Here")
        ratioRecTotal = st.text_input("ratioRecTotal", "Type Here")
        giniSent = st.text_input("giniSent", "Type Here")
        giniRec = st.text_input("giniRec", "Type Here")
        txFreq = st.text_input("txFreq", "Type Here")
        stdBalanceEth = st.text_input("stdBalanceEth", "Type Here")

        result = ""
        if st.button("Predict"):
            result = fraud_prediction([[minTimeBetweenSentTnx, maxTimeBetweenSentTnx, avgTimeBetweenSentTnx, minTimeBetweenRecTnx, maxTimeBetweenRecTnx, avgTimeBetweenRecTnx, lifetime, sentTransactions, receivedTransactions, createdContracts, numUniqSentAddress, numUniqRecAddress, minValSent, maxValSent, avgValSent, minValReceived, maxValReceived, avgValReceived, totalTransactions, totalEtherSent, totalEtherReceived, totalEtherSentContracts, totalEtherBalance, activityDays, dailyMax, ratioRecSent, ratioSentTotal, ratioRecTotal, giniSent, giniRec, txFreq, stdBalanceEth]])
            st.success(result)

if __name__ == '__main__':
    main()

2024-05-26 18:08:55.754 
  command:

    streamlit run C:\Users\Yoyoyo SJ\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2024-05-26 18:08:55.755 Session state does not function when running a script without `streamlit run`
